<a href="https://colab.research.google.com/github/GGoYoungHee/Dacon_koreanNLP/blob/main/XLM_RoBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 주요 라이브러리 설치
!pip install mxnet
!pip install gluonnlp
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 47.3 MB 126 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 7.8 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595731 sha256=146e176c7b12856cbe92e3483a35dc8f5bbeda4a6cafa0cbc6cbd18c27b71356
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 3.5 MB 6.3 MB/s 
     |████████████████████████████████| 596 kB 56.1 MB/s 
     |████████████████████████████████| 67 kB 2.9 MB/s 
     |████████████████████████████████| 895 kB 49.5 MB/s 
     |████████████████████████████████| 6.8 MB 75.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstal

In [ ]:
import pandas as pd
import numpy as np
import os
import gc

import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# seed 값 설정
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import transformers
from transformers import AdamW

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# XLM-RoBERTa 토크나이저를 불러옵니다: https://huggingface.co/xlm-roberta-large
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

MODEL_TYPE = 'xlm-roberta-large'
# 만약 colab pro가 아니면 MODEL_TYPE = 'xlm-roberta-base'를 사용하세요
tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_TYPE)

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513 [00:00<?, ?B/s]

In [ ]:
# XLM-RoBERTa vocab크기 확인
tokenizer.vocab_size

250002

In [ ]:
# XLM-RoBERTa vocab 확인
list(tokenizer.get_vocab())[:10]

['<s>', '<pad>', '</s>', '<unk>', ',', '.', '▁', 's', '▁de', '-']

# Tokenizer 예제

In [ ]:
# 문장 1개 토큰화
MAX_LEN = 10 # 문장의 길이가 max_len보다 크면 안 됩니다. 
# bert의 최대 max length는 512입니다. '안녕하세요'-> len:5 가 아니라, vocab 단어의 토큰 개수 입니다.  '안녕하세요'(107687) -> len:1

sentence1 = '안녕하세요'

encoded_dict = tokenizer.encode_plus(
            sentence1,                
            add_special_tokens = True, 
            truncation=True,
            max_length = MAX_LEN,     
            pad_to_max_length = True, # token_ids 길이를 max_len만큼 맞춰줍니다. 
            return_attention_mask = True,  
            return_tensors = 'pt' # pytorch 텐서를 반환합니다. 
       )


encoded_dict

{'input_ids': tensor([[     0, 107687,      2,      1,      1,      1,      1,      1,      1,
              1]]), 'attention_mask': tensor([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}

In [ ]:
# [EOS](2)이후 [PAD](1)로 max_len만큼 채워져 있습니다. 
# [PAD]는 attention_mask에서 0으로 반환됩니다. 

In [ ]:
input_ids = encoded_dict['input_ids'][0]
att_mask = encoded_dict['attention_mask'][0]

print('input_ids: ', input_ids)
print('att_mask: ', att_mask)

input_ids:  tensor([     0, 107687,      2,      1,      1,      1,      1,      1,      1,
             1])
att_mask:  tensor([1, 1, 1, 0, 0, 0, 0, 0, 0, 0])


In [ ]:
# 문장 2개 토큰화
MAX_LEN = 12 

sentence1 = '안녕하세요'
sentence2 = '데이콘입니다.'

encoded_dict = tokenizer.encode_plus(
            sentence1, sentence2,      
            add_special_tokens = True,
            max_length = MAX_LEN,     
            pad_to_max_length = True,
            return_attention_mask = True,   
            return_tensors = 'pt' 
       )


encoded_dict

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'input_ids': tensor([[     0, 107687,      2,      2,      6,  57991,  58470,   5826,      5,
              2,      1,      1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])}

In [ ]:
input_ids = encoded_dict['input_ids'][0]
att_mask = encoded_dict['attention_mask'][0]

print(input_ids)
print(att_mask)


tensor([     0, 107687,      2,      2,      6,  57991,  58470,   5826,      5,
             2,      1,      1])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0])


In [ ]:
# 토큰 디코드
tokenizer.decode(input_ids,
                skip_special_tokens=False)

# <s>:[BOS], </s>:[CLS],[EOS], <pad>:[PAD]


'<s> 안녕하세요</s></s> 데이콘입니다.</s><pad><pad>'

In [ ]:
#truncation : max_length 길이를 (이어서) 만약 넘어갔다면, 넘어간 이후를 잘라내버리는 방법을 사용
MAX_LEN = 3


sentence = '안녕하세요 히히 나는'

encoded_dict = tokenizer.encode_plus(
            sentence,                
            add_special_tokens = True, 
            truncation=True,
            max_length = MAX_LEN,     
            pad_to_max_length = True, 
            return_attention_mask = True,  
            return_tensors = 'pt' 
       )


encoded_dict


{'input_ids': tensor([[     0, 107687,      2]]), 'attention_mask': tensor([[1, 1, 1]])}

In [ ]:
# '안녕하세요'까지 토큰화가 되었습니다. 
hello=torch.tensor([107687])

tokenizer.decode(hello,
                skip_special_tokens=False)


'안녕하세요'

# XLM-RoBERTa

In [ ]:
!nvidia-smi


Thu Feb 24 08:48:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# device 설정
device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cuda:0


In [ ]:
# 데이터 불러오기
path='/content/drive/MyDrive/Dacon_KoreanNLP/open'
train = pd.read_csv(path+"/train_data.csv")
test = pd.read_csv(path+"/test_data.csv")
submission = pd.read_csv(path+"/sample_submission.csv")

# label 인코딩
label_dict = {"entailment" : 0, "contradiction" : 1, "neutral" : 2}
train['label_num'] = [label_dict[i] for i in train.label]
train.head()

,index,premise,hypothesis,label,label_num
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction,1
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction,1
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment,0
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral,2
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral,2


In [ ]:
from sklearn.model_selection import train_test_split
train_dataset, val_dataset = train_test_split(train, test_size = 0.1)
print(len(train_dataset))
print(len(val_dataset))

22498
2500


In [ ]:
# dataloader에서 오류가 나서 인덱스 재설정
train_dataset.index=[i for i in range(len(train_dataset))]
val_dataset.index=[i for i in range(len(val_dataset))]
val_dataset

,index,premise,hypothesis,label,label_num
0,20890,허블 우주 망원경을 수리하기 위해 우주 공간에서 작업하던 스톤 박사는 폭파된 인공위...,스톤 박사는 우주 공간에 그의 조수와 함께 남겨진다.,contradiction,1
1,21648,한편 같은 날 팔레스타인 측은 이스라엘과 맞닿아 있는 가자지구 접경 지대에서 대규모...,이스라엘에서만 대규모 시위를 벌였다.,contradiction,1
2,4168,또한 기업에 선도적 환경 행정을 추진해 오염물질 발생을 줄이고 쾌적한 환경을 조성하...,오염물질이 발생하지 않는 환경을 조성하려 노력할 것이다.,entailment,0
3,21919,감스트는 리버풀 팬들에게 사과하고 싶다며 큰절을 올렸다.,감스트는 리버풀 팬들에게 사과하고 싶어했다.,entailment,0
4,11278,단언컨데 이영화는 끝나기 10분전이 정말 잼있습니다,이 영화는 끝나기 10분 전까지는 지루합니다.,neutral,2
...,...,...,...,...,...
2495,12188,"정부와 산학연 전문가들이 로봇활용 전략 네트워크를 출범시키고, 전산업의 디지털 전환...",정부와 산학연 전문가들이 전산업의 디지털 전환등을 위해 로봇활용 전략을 마련하기로 했다.,entailment,0
2496,5522,10여 년 만에 출소한 금고 털이범 아버지 정도의 등장으로 아들 봉수와 며느리는 놀란다.,정도에게는 아들이 없다.,contradiction,1
2497,18351,집에 처음 찾아갈때 약간 헷갈릴 수 있습니다.,집에 처음 찾아가도 절대 헤메지 않습니다.,contradiction,1
2498,4971,엔딩에서 뭔가 강한 메시지를 남기는 영화이죠,초반에만 강한 메세지를 남기는 영화이죠.,contradiction,1


# Dataloader 만들기

In [ ]:
# train, val에 사용
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        # 데이터프레임 칼럼 들고오기
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,           
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,   
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        # 숫자로 변환된 label을 텐서로 변환
        target = torch.tensor(self.df_data.loc[index, 'label_num'])
        # input_ids, attention_mask, label을 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask, target)

        return sample

    def __len__(self):
        return len(self.df_data)
    
# test 예측에 사용
class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']


        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,           
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    return_attention_mask = True,   
                    truncation=True,
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        # input_ids, attention_mask를 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask)

        return sample


    def __len__(self):
        return len(self.df_data)


In [ ]:
# 모델 하이퍼파라미터

L_RATE = 1e-5 #1e-3 
MAX_LEN = 256 

BATCH_SIZE = 32 #16 # batch size가 클수록 global minimum에 도달하는 속도가 증가합니다. (GPU 메모리에 따라 변경해 주세요, 너무 크면 OOM 문제가 발생합니다.)
NUM_CORES = os.cpu_count() # Dataloader에 사용됩니다. 

NUM_CORES


4

In [ ]:
from transformers import XLMRobertaForSequenceClassification

model = XLMRobertaForSequenceClassification.from_pretrained(
    MODEL_TYPE, 
    num_labels = 3, # 출력 label의 개수
)

# model을 device위에 올림
model.to(device)

Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.out

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
     

# RoBERTa layer

# 학습

In [ ]:
# optimizer 설정
optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8 
            )


In [ ]:
train_data = CompDataset(train_dataset)
val_data = CompDataset(val_dataset)
test_data = TestDataset(test)

# batch_size 만큼 데이터 분할
train_dataloader = DataLoader(train_data,
                                batch_size=BATCH_SIZE,
                                shuffle=True,
                                num_workers=NUM_CORES)

val_dataloader = DataLoader(val_data,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            num_workers=NUM_CORES)

test_dataloader = DataLoader(test_data,
                                batch_size=BATCH_SIZE,
                                shuffle=False,
                                num_workers=NUM_CORES)



print(len(train_dataloader))
print(len(val_dataloader))
print(len(test_dataloader))

704
79
53


In [ ]:
# 학습 횟수
NUM_EPOCHS=3

# loss값 저장
loss_values = []

# 학습 시작
for epoch in range(NUM_EPOCHS):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    
    stacked_val_labels = []
    targets_list = []

    # ========================================
    #               Training
    # ========================================
    
    print('Training...')
    
    # train mode 변환
    model.train()
    # True로 설정하게 되면 해당 텐서에서 어떤 연산이 이루어졌는지 추적할 수 있고, 해당 텐서에 대한 그라디언트를 저장하게 됩니다. 
    torch.set_grad_enabled(True)


    # 1epoch마다 loss값 초기화
    total_train_loss = 0

    for i, batch in enumerate(train_dataloader):
        
        train_status = 'Batch ' + str(i) + ' of ' + str(len(train_dataloader))
        
        print(train_status, end='\r')


        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        # 3개의 인풋
        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask,
                    labels=b_labels)
        
        # outputs tuple: (loss, logits)
        loss = outputs[0]
        
        # loss는 텐서이기 때문에 숫자로 변환 후 더합니다. 
        total_train_loss = total_train_loss + loss.item()
        
        # backward()를 하기 전에 optimizer의 그라디언트를 0으로 합니다. 
        optimizer.zero_grad()
        
        # 그라디언트 계산
        loss.backward()
        
        
        # "exploding gradients" 문제를 예방해줍니다.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        
        # optimizer 가중치 업데이트
        optimizer.step() 
    
    print('Train loss:' ,total_train_loss)


    # ========================================
    #               Validation
    # ========================================
    
    print('\nValidation...')

    # evaluation mode로 변환
    model.eval()

    # validation 과정에서는 그라디언트를 연산하거나 저장하지 않습니다.(메모리, 진행 속도 세이브)
    torch.set_grad_enabled(False)
    
    total_val_loss = 0
    

    for j, batch in enumerate(val_dataloader):
        
        val_status = 'Batch ' + str(j) + ' of ' + str(len(val_dataloader))
        
        print(val_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)      


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask, 
                labels=b_labels)
        
        loss = outputs[0]
        
        total_val_loss = total_val_loss + loss.item()
        

        # 예측값
        preds = outputs[1]


        # 예측값을 CPU로 이동시킵니다. 
        val_preds = preds.detach().cpu().numpy()
        
        # labels을 cpu로 이동시킵니다.
        targets_np = b_labels.to('cpu').numpy()

        targets_list.extend(targets_np)

        if j == 0:  # 첫 번째 batch일 떄
            stacked_val_preds = val_preds

        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

        

    
    # validation accuracy 계산
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    
    val_acc = accuracy_score(y_true, y_pred)
    
    
    print('Val loss:' ,total_val_loss)
    print('Val acc: ', val_acc)


    # 모델 저장
    torch.save(model.state_dict(), 'epoch:{}_model.pt'.format(epoch))
    
    # 메모리 관리
    gc.collect()


======== Epoch 1 / 3 ========
Training...


RuntimeError: ignored

In [ ]:
# 모델 불러오기 (선택)
from transformers import XLMRobertaForSequenceClassification

model = XLMRobertaForSequenceClassification.from_pretrained(
    MODEL_TYPE, 
    num_labels = 3, 
)

model.to(device)
model.load_state_dict(torch.load('/content/epoch:1_model.pt'))

In [ ]:
# test 시작
for j, batch in enumerate(test_dataloader):
        
        inference_status = 'Batch ' + str(j+1) + ' of ' + str(len(test_dataloader))
        
        print(inference_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask)
        
        # 예측값
        preds = outputs[0]
        preds = preds.detach().cpu().numpy()

        
        if j == 0:  
            stacked_preds = preds

        else:
            stacked_preds = np.vstack((stacked_preds, preds))


In [ ]:
stacked_preds

In [ ]:
# argmax로 label결정
preds = np.argmax(stacked_preds, axis=1)

preds

In [ ]:
# 숫자 label을 다시 영어로 변환
la=[]
for i in preds:
  if i == 0:
    la.append("entailment")
  elif i == 1:
    la.append("contradiction")
  elif i == 2:
    la.append('neutral')

submission['label']=la

In [ ]:
submission.to_csv("/content/drive/MyDrive/Dacon_KoreanNLP/open/sample_submission-XLMRoBERTa.csv", index = False)